# Disen 3 to Model

In [1]:
import os
import time
import pickle
import pandas as pd
import numpy as np

datasets_name = 'Sports_and_Outdoors'
data_path = 'dict/' + datasets_name +'/session_data.csv'

data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID']]


num_pos = 10
num_neg = 10

# dict (sessionID:[itemID,itemID])
sess_all = {}

for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() 

for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
    else:
        tes_sess[sess_temp] = all_seqs


item_dict = {} #dict(old_itemID: new_itemID)



# tra_sess tra_price tra_cate
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_seqs = []
    global item_ctr
    item_ctr = 1
    for s in tra_sess:
        seq = tra_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            print('session length is 1')
            continue
        train_seqs += [outseq]
    print("#train_session",len(train_seqs))
    print("#train_items",item_ctr-1)
    return train_seqs


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    for s in tes_sess:
        outseq = []
        for i in tes_sess[s]:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            print('obtain test session length is 1')
            continue
        test_seqs += [outseq]
    print("#test_session: ",len(test_seqs))
    return test_seqs

# Convert test sessions to sequences, renumbering items that do not appear in training set
def obtian_tes_cold():
    test_seqs = []
    num_cold = 0
    global item_ctr
#     test_cate = []
    for s in tes_sess:
        outseq = []
        contain_cold = False
        for i in tes_sess[s]:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                num_cold += 1
                contain_cold = True
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1

        if len(outseq) < 2:
            print('obtain test session length is 1')
            continue
        if contain_cold:
            test_seqs_cold += [outseq]
        else:
            test_seqs += [outseq]
    print("#test_session: ",len(test_seqs))
    print("#cold_test_session: ",len(test_seqs_cold))
    print('#cold_items',str(num_cold))
    return test_seqs, test_seqs_cold


# 数据增强
def process_seqs(iseqs, iprice, icate):
    out_seqs = []
    out_price = []
    out_cate = []
    labs = []
    max_length = 19
    for seq, pri, rat, cat in zip(iseqs, iprice, icate):
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
            out_price += [pri[:-i]]
            out_cate += [cat[:-i]]
    return out_seqs, out_price, out_cate, labs

# 无数据增强
def process_seqs_no(iseqs):
    print("no data augment")
    out_seqs = []
    labs = []
    max_length = 19
    for seq in iseqs:     
        labs += [seq[-1]]
        out_seqs += [seq[:-1]]
    return out_seqs, labs

tra_seqs = obtian_tra()
tes_seqs  = obtian_tes()
old2new_nocold_path = './dict/' + datasets_name + '/old2newNocold.csv'
old_list_nocold = list(item_dict.keys())
new_list_nocold = list(item_dict.values())
old2new_nocold_dict = {'old':old_list_nocold, 'new':new_list_nocold}
data_old_new_no = pd.DataFrame(old2new_nocold_dict)
data_old_new_no.to_csv(old2new_nocold_path)


tr_seqs,  tr_labs = process_seqs_no(tra_seqs)
te_seqs,  te_labs = process_seqs_no(tes_seqs)


print('train sequence: ',tr_seqs[:5])
print('train lab: ',tr_labs[:5])


# 增加 all_item_price_list & all_item_rating_list

# 共现关系
fre_seq = tra_seqs + tes_seqs
fre_mat = np.zeros((item_ctr-1, item_ctr-1), dtype=int)
for sess in fre_seq:
    for it1 in sess:
        for it2 in sess:
            if it1 != it2:
                fre_mat[it1-1, it2-1] +=1 


fre_mat_norm = fre_mat/(np.sum(fre_mat, axis= 1)+1e-8)[:, np.newaxis]


indices = []
all_data = []
all_item_pair = []
non_co = 0
for i_list in fre_mat_norm:
    i_temp_list = np.nonzero(i_list)[0].tolist()
    if len(i_temp_list) == 0:
        non_co += 1
    indices += i_temp_list
    all_data += i_list[np.nonzero(i_list)].tolist()
    all_item_pair.append(i_temp_list)
print('item without co-occurren: ',str(non_co))
indptr = []
indptr.append(0)
for j in range(len(all_item_pair)):
    session = np.unique(all_item_pair[j]) #统计session中不同item，去重，并按照item_id排序
    length = len(session)
    s = indptr[-1]
    indptr.append((s + length))
# indptr:session长度累加和; indices:item_id 减1, 由每个session内item组成; data:item在session内的权重，全部为1.
coo_mat = (all_data, indices, indptr)


indentify_mat = np.identity(fre_mat.shape[0],dtype=int)
fre_mat = fre_mat - indentify_mat
# 每行内，从大到小排序
arg_mat = np.argsort(-fre_mat, axis = 1)
pos_count =  np.sort(-fre_mat, axis=1)[:, :num_pos].tolist()

# 共现最多与最少的序号，1序
pos_index = (arg_mat[:, :num_pos] + 1).tolist()

neg_index = (arg_mat[:, -num_neg-1:-1] + 1).tolist()

pos_neg_list = []
pos_neg_list.append(pos_index)
pos_neg_list.append(neg_index)
pos_neg_list.append(pos_count)

item_pairs_dict = {}
for item_id, idx_list, num_list in zip(range(1,item_ctr), pos_index, pos_count):
    if item_id not in item_pairs_dict:
        item_pairs_dict[item_id] = []
    for idx_i, num_i in zip(idx_list, num_list):
        if num_i != 0:
            item_pairs_dict[item_id].append(idx_i)

tr_flag = []
for sess_temp, lab_temp in zip(tr_seqs, tr_labs):
    flag = -1
    for ix_temp in list(set(sess_temp)):
        if lab_temp in item_pairs_dict[ix_temp]:
            flag = 1
    tr_flag.append(flag)
print('id, train, co-occurence sess: ',str(tr_flag.count(1)))
print('Modality, train, non co-occurence sess: ',str(tr_flag.count(-1)))


te_flag = []
for sess_temp, lab_temp in zip(te_seqs, te_labs):
    flag = 0
    for ix_temp in list(set(sess_temp)):
        if lab_temp in item_pairs_dict[ix_temp]:
            flag = 1
    te_flag.append(flag)
print('id, test, co-occurence sess: ',str(tr_flag.count(1)))
print('Modality, test, non co-occurence sess: ',str(tr_flag.count(-1)))

tra = (tr_seqs, tr_flag, tr_labs, coo_mat)
tes = (te_seqs, te_flag, te_labs, coo_mat)


all_train = 0
all_tes = 0
for seq in tra_seqs:
    all_train += len(seq)
for seq in tes_seqs:
    all_tes += len(seq)
print('#train_interactions: ',all_train)
print('#test_interactions: ',all_tes)


print('#train_session: ',(len(tra_seqs)))
print('#test_session: ',(len(tes_seqs)))


print('sequence average length: ', (all_train+all_tes)/(len(tra_seqs) + len(tes_seqs) * 1.0))
train_data_path = './datasets/' + datasets_name



if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_normal_test = train_data_path + "/test.txt"
path_frequence = train_data_path + "/frequence.txt"

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_normal_test, 'wb'))
pickle.dump(pos_neg_list, open(path_frequence, 'wb'))


# get asin item_id 获得asin list 有顺序, 给处理 img及text
asin2old_path = 'dict/' + datasets_name +'/asin2itemID.csv'
old2new_path = 'dict/' + datasets_name +'/old2newNocold.csv'


asin2old = pd.read_csv(asin2old_path)
asin2old = asin2old.rename(columns={ 'itemID':'old'})
asin2old = asin2old[['asin', 'old']]


old2new = pd.read_csv(old2new_path)
old2new = old2new[['old', 'new']]


asin2new = pd.merge(asin2old, old2new, how='left', on = 'old')


asin2new = asin2new.dropna(axis=0)
asin2ItemID = asin2new.reset_index()[['asin', 'new']]

asin2ItemID.sort_values(by=["new"],inplace=True,ascending=[True])


asin_list = asin2ItemID['asin'].tolist()
asin_list_save = train_data_path + '/asinlist.npy'
np.save(asin_list_save, asin_list)



print("dataset: ", datasets_name)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("data formulation: id_seq,labs")
print("done")



#train_session 81516
#train_items 14650
#test_session:  8976
no data augment
no data augment
train sequence:  [[1], [3], [4], [6, 7, 8], [10, 11, 12, 13, 14, 15, 16]]
train lab:  [2, 3, 5, 9, 17]
item without co-occurren:  85
id, train, co-occurence sess:  38771
Modality, train, non co-occurence sess:  42745
id, test, co-occurence sess:  38771
Modality, test, non co-occurence sess:  42745
#train_interactions:  253927
#test_interactions:  28175
#train_session:  81516
#test_session:  8976
sequence average length:  3.1174247447288157
dataset:  Sports_and_Outdoors
2023-08-20 18:33:34
data formulation: id_seq,labs
done


### NARM, S-POP, SR-GNN, DHCN, AttenMixer 

In [2]:
import os
import pickle
import time
import pandas as pd
import numpy as np


datasets_name = 'Sports_and_Outdoors'
data_path = 'dict/' + datasets_name +'/session_data.csv'


data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID']]


# dict (sessionID:[itemID,itemID])
sess_all = {}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() # dict(session_id:[item_id,item_id])
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
    else:
        tes_sess[sess_temp] = all_seqs

item_dict = {}#dict(old_itemID: new_itemID)
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_seqs = []
    item_ctr = 1
    for s in tra_sess:
        seq = tra_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_seqs += [outseq]
    print("#train_items",item_ctr)    
    return train_seqs


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    for s in tes_sess:
        seq = tes_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            continue
        test_seqs += [outseq]
    return test_seqs

# 数据增强
def process_seqs(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
    return out_seqs, labs

# 无数据增强
def process_seqs_no(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        labs += [seq[-1]]
        out_seqs += [seq[:-1]]
    return out_seqs, labs


tra_seqs = obtian_tra()
tes_seqs = obtian_tes()

tr_seqs, tr_labs = process_seqs_no(tra_seqs)
te_seqs, te_labs = process_seqs_no(tes_seqs)

tra = (tr_seqs, tr_labs)
tes = (te_seqs, te_labs)
print("train sequence: ", len(tr_seqs))
print("test sequence: ", len(te_seqs))
print("train sample: ", tr_seqs[:3], tr_labs[:3])
print("test sample: ", te_seqs[:3], te_labs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('sequence average length: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))

train_data_path = 'NARM/' + datasets_name

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_data_test = train_data_path + "/test.txt"

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_data_test, 'wb'))
print("dataset: ", datasets_name)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

#train_items 14651
train sequence:  81516
test sequence:  8976
train sample:  [[1], [3], [4]] [2, 3, 5]
test sample:  [[6842], [4504, 3161, 3161, 676], [677]] [9767, 9087, 677]
sequence average length:  3.1174247447288157
dataset:  Sports_and_Outdoors
2023-08-20 18:48:12
done


## COTREC & DGNN

In [1]:
import os
import pickle
import time
import pandas as pd
import numpy as np

datasets_name = 'Sports_and_Outdoors'
data_path = 'dict/' + datasets_name +'/session_data.csv'


data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID']]


# dict (sessionID:[itemID,itemID])
sess_all = {}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() # dict(session_id:[item_id,item_id])
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
    else:
        tes_sess[sess_temp] = all_seqs

item_dict = {}#dict(old_itemID: new_itemID)
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_seqs = []
    item_ctr = 1
    for s in tra_sess:
        seq = tra_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_seqs += [outseq]
    print("#train_items",item_ctr)    
    return train_seqs


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    for s in tes_sess:
        seq = tes_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            continue
        test_seqs += [outseq]
    return test_seqs

# 数据增强
def process_seqs(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
    return out_seqs, labs

# 无数据增强
def process_seqs_no(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        labs += [seq[-1]]
        out_seqs += [seq[:-1]]
    return out_seqs, labs

tra_seqs = obtian_tra()
tes_seqs = obtian_tes()

all_train_seqs = tra_seqs + tes_seqs

tr_seqs, tr_labs = process_seqs_no(tra_seqs)
te_seqs, te_labs = process_seqs_no(tes_seqs)

tra = (tr_seqs, tr_labs)
tes = (te_seqs, te_labs)
print("train sequence: ", len(tr_seqs))
print("test sequence: ", len(te_seqs))
print("train sample: ", tr_seqs[:3], tr_labs[:3])
print("test sample: ", te_seqs[:3], te_labs[:3])
print("all train sequences:", all_train_seqs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('sequence average length: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))

train_data_path = 'COTREC&DGNN/' + datasets_name + 'Disen'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_data_test = train_data_path + "/test.txt"
path_data_all = train_data_path+ "/all_train_seq.txt"

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_data_test, 'wb'))
pickle.dump(all_train_seqs, open(path_data_all, 'wb'))
print("dataset: ", datasets_name)
print("COTREC&DGNN")
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))

#train_items 14651
train sequence:  81516
test sequence:  8976
train sample:  [[1], [3], [4]] [2, 3, 5]
test sample:  [[6842], [4504, 3161, 3161, 676], [677]] [9767, 9087, 677]
all train sequences: [[1, 2], [3, 3], [4, 5]]
sequence average length:  3.1174247447288157
dataset:  Sports_and_Outdoors
COTREC&DGNN
2023-08-20 18:53:41


## LESSR & MSGIFSR

In [1]:
import os
import time
import pickle
import pandas as pd
import numpy as np


datasets_name = 'Sports_and_Outdoors'
# data_path = 'data_time_interval/' + datasets_name +'_data.csv'
data_path = 'dict/' + datasets_name +'/session_data.csv'

data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID']]


# dict (sessionID:[itemID,itemID])
sess_all = {}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() # dict(session_id:[item_id,item_id])
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
    else:
        tes_sess[sess_temp] = all_seqs

item_dict = {}#dict(old_itemID: new_itemID)
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_seqs = []
    item_ctr = 1
    for s in tra_sess:
        seq = tra_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_seqs += [outseq]
    print("#train_items",item_ctr)    
    return train_seqs


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    for s in tes_sess:
        seq = tes_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            continue
        test_seqs += [outseq]
    return test_seqs


tra_seqs = obtian_tra()
tes_seqs = obtian_tes()

print("train sequence: ", len(tra_seqs))
print("test sequence: ", len(tes_seqs))
print("train sample: ", tra_seqs[:3])
print("test sample: ", tes_seqs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('sequence average length: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))

# train_data_path = 'LESSR/Cell_Phones_and_Accessories'
train_data_path = 'LESSRDisen/' + datasets_name + 'Disen'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_data_test = train_data_path + "/test.txt"
path_item_num = train_data_path + '/num_items.txt'

print('save item_num :', len(item_dict))
pickle.dump(len(item_dict)+1, open(path_item_num, 'wb'))
pickle.dump(tra_seqs, open(path_data_train, 'wb'))
pickle.dump(tes_seqs, open(path_data_test, 'wb'))
print("dataset: ", datasets_name)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

#train_items 14651
train sequence:  81516
test sequence:  8976
train sample:  [[1, 2], [3, 3], [4, 5]]
test sample:  [[6842, 9767], [4504, 3161, 3161, 676, 9087], [677, 677]]
sequence average length:  3.1174247447288157
save item_num : 14650
dataset:  Sports_and_Outdoors
2023-08-20 18:54:40
done


## BERT4Rec

In [1]:
import os
import pickle
import time
import pandas as pd
import numpy as np


datasets_name = 'Sports_and_Outdoors'
# data_path = 'data_time_interval/' + datasets_name +'_data.csv'
data_path = 'dict/' + datasets_name +'/session_data.csv'


data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID']]


# dict (sessionID:[itemID,itemID])
sess_all = {}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)

sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() # dict(session_id:[item_id,item_id])
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
    else:
        tes_sess[sess_temp] = all_seqs

item_dict = {}#dict(old_itemID: new_itemID)
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_seqs = []
    sessid_seqs = []
    item_ctr = 1
    for s in tra_sess:
        seq = tra_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_seqs += [outseq]
        sessid_seqs += [s]
    print("#train_items",item_ctr)    
    return train_seqs, sessid_seqs


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    test_sessid_seqs = []
    for s in tes_sess:
        seq = tes_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            continue
        test_seqs += [outseq]
        test_sessid_seqs += [s]
    return test_seqs, test_sessid_seqs

# 数据增强
def process_seqs(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
    return out_seqs, labs

# 无数据增强
def process_seqs_no(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        labs += [seq[-1]]
        out_seqs += [seq[:-1]]
    return out_seqs, labs


tra_seqs, tra_sessid = obtian_tra()
tes_seqs, tes_sessid = obtian_tes()

tr_seqs, tr_labs = process_seqs_no(tra_seqs)
te_seqs, te_labs = process_seqs_no(tes_seqs)

tra = (tra_sessid, tr_seqs, tr_labs)
tes = (tes_sessid, te_seqs, te_labs)
print("train sequence: ", len(tr_seqs))
print("test sequence: ", len(te_seqs))
print("train sample: ", tr_seqs[:3], tr_labs[:3])
print("test sample: ", te_seqs[:3], te_labs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('sequence average length: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))

train_data_path = 'Bert4RecDisen/' + datasets_name + 'Disen'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_data_test = train_data_path + "/test.txt"

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_data_test, 'wb'))
print("dataset: ", datasets_name)
print("BERT4Rec")
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

#train_items 14651
train sequence:  81516
test sequence:  8976
train sample:  [[1], [3], [4]] [2, 3, 5]
test sample:  [[6842], [4504, 3161, 3161, 676], [677]] [9767, 9087, 677]
sequence average length:  3.1174247447288157
dataset:  Sports_and_Outdoors
2023-08-20 18:55:19
done


## SKNN

In [2]:
import os
import pickle
import time
import pandas as pd
import numpy as np

datasets_name = 'Sports_and_Outdoors'
# data_path = 'data_time_interval/' + datasets_name +'_data.csv'
data_path = 'dict/' + datasets_name +'/session_data.csv'


print("******SKNN******")

data_all = pd.read_csv(data_path)
data = data_all[['sessionID', 'itemID', 'time']]

 

# dict (sessionID:[itemID,itemID])
sess_all = {}
sess_date = {}
for _, row in data.iterrows():
    sess_id = row['sessionID']
    item_id = row['itemID']
    timestamp = row['time']
    if sess_id in sess_all:
        sess_all[sess_id].append(item_id)
        sess_date[sess_id] = timestamp
    else:
        sess_all[sess_id] = []
        sess_all[sess_id].append(item_id)
        sess_date[sess_id] = timestamp


sess_total = data['sessionID'].max()
split_num = int(sess_total/10*9)

tra_sess = dict() # dict(session_id:[item_id,item_id])
tes_sess = dict() # dict(session_id:[item_id,item_id])
for sess_temp in sess_all.keys():
    all_seqs = sess_all[sess_temp]
    if len(all_seqs) < 2:
        continue
    if len(all_seqs) > 20:
        all_seqs = all_seqs[:20]
    if int(sess_temp) < split_num:
        tra_sess[sess_temp] = all_seqs
    else:
        tes_sess[sess_temp] = all_seqs


item_dict = {}#dict(old_itemID: new_itemID)
# Convert training sessions to sequences and renumber items to start from 1
def obtian_tra():
    train_seqs = []
    sessid_seqs = []
    time_seqs = []
    item_ctr = 1
    for s in tra_sess:
        seq = tra_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
            else:
                outseq += [item_ctr]
                item_dict[i] = item_ctr
                item_ctr += 1
        if len(outseq) < 2:  # Doesn't occur
            continue
        train_seqs += [outseq]
        sessid_seqs += [s]
        time_seqs += [sess_date[s]]
    print("#train_items",item_ctr)    
    return train_seqs, sessid_seqs, time_seqs


# Convert test sessions to sequences, ignoring items that do not appear in training set
def obtian_tes():
    test_seqs = []
    test_sessid_seqs = []
    test_time_seqs = []
    for s in tes_sess:
        seq = tes_sess[s]
        outseq = []
        for i in seq:
            if i in item_dict:
                outseq += [item_dict[i]]
        if len(outseq) < 2:
            continue
        test_seqs += [outseq]
        test_sessid_seqs += [s]
        test_time_seqs += [sess_date[s]]
    return test_seqs, test_sessid_seqs, test_time_seqs

# 数据增强
def process_seqs(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
    return out_seqs, labs

# 无数据增强
def process_seqs_no(iseqs):
    out_seqs = []
    labs = []
    for seq in iseqs:
        labs += [seq[-1]]
        out_seqs += [seq[:-1]]
    return out_seqs, labs


tra_seqs, tra_sessid, tra_time = obtian_tra()
tes_seqs, tes_sessid, tes_time = obtian_tes()

tr_seqs, tr_labs = process_seqs_no(tra_seqs)
te_seqs, te_labs = process_seqs_no(tes_seqs)

tra = (tra_sessid, tr_seqs, tr_labs, tra_time)
tes = (tes_sessid, te_seqs, te_labs, tes_time)
print("train sequence: ", len(tr_seqs))
print("test sequence: ", len(te_seqs))
print("train sample: ", tr_seqs[:3], tr_labs[:3])
print("test sample: ", te_seqs[:3], te_labs[:3])
all = 0

for seq in tra_seqs:
    all += len(seq)
for seq in tes_seqs:
    all += len(seq)
print('sequence average length: ', all/(len(tra_seqs) + len(tes_seqs) * 1.0))

train_data_path = 'SKNNDisen/' + datasets_name

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)
path_data_train = train_data_path + "/train.txt"
path_data_test = train_data_path + "/test.txt"

pickle.dump(tra, open(path_data_train, 'wb'))
pickle.dump(tes, open(path_data_test, 'wb'))
print("dataset: ", datasets_name)
print(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()))
print("done")

******SKNN******
#train_items 14651
train sequence:  81516
test sequence:  8976
train sample:  [[1], [3], [4]] [2, 3, 5]
test sample:  [[6842], [4504, 3161, 3161, 676], [677]] [9767, 9087, 677]
sequence average length:  3.1174247447288157
dataset:  Sports_and_Outdoors
2023-08-20 18:56:38
done
